In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from scipy.stats.mstats import winsorize

In [3]:
# Read data
data_raw = pd.read_csv('./dataset_raw_v2.csv')
weekly_equity_returns = pd.read_csv('./weekly_equity_returns.csv')

In [4]:
# construct dataframe to store results
Return_variable = pd.DataFrame()
Return_variable['gvkey'] = data_raw['gvkey']
Return_variable['fyear'] = data_raw['fyear']

## Calculate the variables which only related to dataset data_raw

### Calculate Log_scope_1

In [5]:
# construct the dataframe to store Emission data
Emission_table = pd.DataFrame()
Emission_table['gvkey'] = data_raw['gvkey']
Emission_table['fyear'] = data_raw['fyear']

In [6]:
Emission_table['log_scope_1'] = np.log(data_raw['Scope_1'])

c:\Users\13976\.conda\envs\tf\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [7]:
# Replace inf by using NaN
Emission_table['log_scope_1'].replace(-float('inf'), np.nan, inplace=True)

# Drop NaN
Emission_table['log_scope_1'] = Emission_table['log_scope_1'].dropna()

### Calculate Log_scope_2

In [8]:
Emission_table['log_scope_2'] = np.log(data_raw['Scope_2'])

c:\Users\13976\.conda\envs\tf\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [9]:
# Replace inf by using NaN
Emission_table['log_scope_2'].replace(-float('inf'), np.nan, inplace=True)

# Drop NaN
Emission_table['log_scope_1'] = Emission_table['log_scope_1'].dropna()

### Calculate Growth Rate in Carbon Emissions scope 1

In [10]:
# Sort data by 'gvkey' and 'fyear'
data_raw = data_raw.sort_values(by=['gvkey', 'fyear'])

# Calculate growth rate for previous year
data_raw['Growth Rate in Carbon Emissions Scope 1'] = data_raw.groupby('gvkey')['Scope_1'].pct_change()

In [11]:
# Replace inf by using NaN
growth_rates_scope_1 = data_raw[['gvkey', 'fyear', 'Growth Rate in Carbon Emissions Scope 1']].copy()
growth_rates_scope_1['Growth Rate in Carbon Emissions Scope 1'].replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop NaN
growth_rates_scope_1 = growth_rates_scope_1.dropna()

# Winsorised the data in 2.5%
winsorised_growth_rates_scope_1 = winsorize(growth_rates_scope_1['Growth Rate in Carbon Emissions Scope 1'], limits=[0.0125, 0.0125])

# Add winsorised data
growth_rates_scope_1['Winsorised Growth Rate in Carbon Emissions Scope 1'] = winsorised_growth_rates_scope_1

In [12]:
# Merge winsorised data to emission table on 'gvkey' and 'fyear'
Emission_table = Emission_table.merge(
    growth_rates_scope_1[['gvkey', 'fyear', 'Winsorised Growth Rate in Carbon Emissions Scope 1']],
    on=['gvkey', 'fyear'],
    how='left'
)

### Calculate Growth Rate in Carbon Emissions scope 2

In [13]:
# Sort data by 'gvkey' and 'fyear'
data_raw = data_raw.sort_values(by=['gvkey', 'fyear'])

# Calculate growth rate for previous year
data_raw['Growth Rate in Carbon Emissions Scope 2'] = data_raw.groupby('gvkey')['Scope_2'].pct_change()

In [14]:
# Replace inf by using NaN
growth_rates_scope_2 = data_raw[['gvkey', 'fyear', 'Growth Rate in Carbon Emissions Scope 2']].copy()
growth_rates_scope_2['Growth Rate in Carbon Emissions Scope 2'].replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop NaN
growth_rates_scope_2 = growth_rates_scope_2.dropna()

# Winsorised the data in 2.5%
winsorised_growth_rates_scope_2 = winsorize(growth_rates_scope_2['Growth Rate in Carbon Emissions Scope 2'], limits=[0.0125, 0.0125])

# Add winsorised data
growth_rates_scope_2['Winsorised Growth Rate in Carbon Emissions Scope 2'] = winsorised_growth_rates_scope_2

In [15]:
# Merge winsorised data to emission table on 'gvkey' and 'fyear'
Emission_table = Emission_table.merge(
    growth_rates_scope_2[['gvkey', 'fyear', 'Winsorised Growth Rate in Carbon Emissions Scope 2']],
    on=['gvkey', 'fyear'],
    how='left'
)

### Calculate Carbon Intensity Scope 1

In [16]:
# Carbon intensity is calculated as the total revenue diveded by the total carbon emission quantity.
Emission_table['Carbon Intensity Scope 1'] = data_raw['Scope_1'] / data_raw['revt'] / 100

In [17]:
# Replace inf by using NaN
Emission_table['Carbon Intensity Scope 1'].replace([np.inf, -np.inf], np.nan, inplace=True)

# Winsorise the carbon intensity
winsorised_carbon_intensity_scope1 = winsorize(Emission_table['Carbon Intensity Scope 1'].dropna(), limits=[0.0125, 0.0125])

Emission_table.loc[Emission_table['Carbon Intensity Scope 1'].notna(), 'Winsorised Carbon Intensity Scope 1'] = winsorised_carbon_intensity_scope1

### Calculate Carbon Intensity Scope 2

In [18]:
# Carbon intensity is calculated as the total revenue diveded by the total carbon emission quantity.
Emission_table['Carbon Intensity Scope 2'] = data_raw['Scope_2'] / data_raw['revt'] / 100

In [19]:
# Replace inf by using NaN
Emission_table['Carbon Intensity Scope 2'].replace([np.inf, -np.inf], np.nan, inplace=True)

# Winsorise the carbon intensity
winsorised_carbon_intensity_scope2 = winsorize(Emission_table['Carbon Intensity Scope 2'].dropna(), limits=[0.0125, 0.0125])

Emission_table.loc[Emission_table['Carbon Intensity Scope 2'].notna(), 'Winsorised Carbon Intensity Scope 2'] = winsorised_carbon_intensity_scope2

In [20]:
# Carbon intensity direct is generally equal to the carbon intensity scope 1, and carbon intensity indirect is the sum of the carbon intensity of scope 2 and
# the carbon intensity of scope 3. However, we don't have the data related to scope 3, we cannot find out the carbon intensity indirect. Also, we cannot calculate
# the GHG direct impact retio and GHG indirect impact ratio due to the lack of data about emission scope 3.

In [21]:
Emission_table.describe()

,gvkey,fyear,log_scope_1,log_scope_2,Winsorised Growth Rate in Carbon Emissions Scope 1,Winsorised Growth Rate in Carbon Emissions Scope 2,Carbon Intensity Scope 1,Winsorised Carbon Intensity Scope 1,Carbon Intensity Scope 2,Winsorised Carbon Intensity Scope 2
count,16347.000000,16347.000000,16340.000000,16342.000000,13721.000000,13721.000000,15999.000000,15999.000000,15999.000000,15999.000000
mean,55208.429375,2016.562794,9.910894,10.126252,0.069777,0.085642,1.676346,1.428081,0.364424,0.314744
std,64178.011324,2.857374,2.991095,2.483599,0.378345,0.392911,7.260993,5.138599,1.687184,0.427828
min,1004.000000,2010.000000,-1.645065,-0.079043,-0.766253,-0.676186,0.000000,0.002459,0.000000,0.006984
25%,9465.000000,2015.000000,7.901009,8.645540,-0.069214,-0.073421,0.035631,0.035631,0.073874,0.073874
50%,24216.000000,2017.000000,9.856206,10.326218,0.020804,0.022085,0.129947,0.129947,0.167862,0.167862
75%,66588.000000,2019.000000,11.699423,11.822373,0.131443,0.153653,0.293556,0.293556,0.401159,0.401159
max,328795.000000,2020.000000,18.804073,16.920606,2.102875,2.270040,196.809468,34.648542,141.591432,2.519237


### Calculate LOGSIZE

In [22]:
# mkvalt is the market capitablization
Return_variable['LOGSIZE'] = np.log(data_raw['mkvalt'])

# Replace inf by using NaN
Return_variable['LOGSIZE'].replace([np.inf, -np.inf], np.nan, inplace=True)

### Calculate B/M

In [23]:
# B/M is the book value of equity divided by market value of equity. Book value of equity is total asset minus total liabilities.
# It is clear that at is total asset and lt is liabilities.
Return_variable['B/M'] = (data_raw['at'] - data_raw['lt']) / data_raw['mkvalt']

# Replace inf by using NaN
Return_variable['B/M'].replace([np.inf, -np.inf], np.nan, inplace=True)

# Winsorise the B/M
winsorised_BM = winsorize(Return_variable['B/M'].dropna(), limits=[0.0125, 0.0125])
Return_variable.loc[Return_variable['B/M'].notna(), 'Winsorised BM'] = winsorised_BM

### Calculate Leverage

In [24]:
# Leverage is the bool value of leverage defined as the book value of debt divided by the book value of assets.
Return_variable['Leverage'] = ( data_raw['dlc'] + data_raw['dltt'] )/ data_raw['at']

# Replace inf by using NaN
Return_variable['Leverage'].replace([np.inf, -np.inf], np.nan, inplace=True)

# Winsorise the Leverage
winsorised_Leverge = winsorize(Return_variable['Leverage'].dropna(), limits=[0.0125, 0.0125])
Return_variable.loc[Return_variable['Leverage'].notna(), 'Winsorised Leverage'] = winsorised_Leverge

### Calcualte INVEST/A

In [25]:
# INVEST/A is the CAPEX divided by book value of assets, where CAPEX is the capital expenditure.
Return_variable['INVEST/A'] = data_raw['capx'] / data_raw['at']

# Replace inf by using NaN
Return_variable['INVEST/A'].replace([np.inf, -np.inf], np.nan, inplace=True)

# Winsorise the INVEST/A
winsorised_InvestA = winsorize(Return_variable['INVEST/A'].dropna(), limits=[0.0125, 0.0125])
Return_variable.loc[Return_variable['INVEST/A'].notna(), 'Winsorised INVESTA'] = winsorised_InvestA

### Calculate ROE

In [26]:
# ROE is return of equity, which is calculated by net earning divides the stockholders' equity.
Return_variable['ROE'] = (data_raw['ni'] / data_raw['ceq']) * 100

# Replace inf by using NaN
Return_variable['ROE'].replace([np.inf, -np.inf], np.nan, inplace=True)

# Winsorise the ROE
winsorised_ROE = winsorize(Return_variable['ROE'].dropna(), limits=[0.0125, 0.0125])
Return_variable.loc[Return_variable['ROE'].notna(), 'Winsorised ROE'] = winsorised_ROE

### Calculate LOGPPE

In [27]:
# LOGPPE is the natural logarithm of plant, property & equipment (in $ million).
Return_variable['LOGPPE'] = np.log(data_raw['ppegt'])

# Replace inf by using NaN
Return_variable['LOGPPE'].replace([np.inf, -np.inf], np.nan, inplace=True)

c:\Users\13976\.conda\envs\tf\lib\site-packages\pandas\core\arraylike.py:402: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


### Calculate SALESGR

In [28]:
# Sort by using gvkey and fyear
data_raw = data_raw.sort_values(by=['gvkey', 'fyear'])

# Calculate the growth rate
data_raw['SALESGR'] = data_raw.groupby('gvkey')['sale'].pct_change()
SALESGR1 = data_raw[['gvkey', 'fyear', 'SALESGR']].copy()

# Replace inf by using NaN
SALESGR1.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop NaN
SALESGR1.dropna()

# Winsorised the data in 2.5%
winsorised_SALESGR = winsorize(SALESGR1['SALESGR'], limits=[0.0025, 0.0025])

# Add winsorised data
SALESGR1['Winsorised SALESGR'] = winsorised_SALESGR

In [29]:
# merge winsorised data to emission table on 'gvkey' and 'fyear'
Return_variable = Return_variable.merge(
    SALESGR1[['gvkey', 'fyear', 'Winsorised SALESGR']],
    on=['gvkey', 'fyear'],
    how='left'
)

In [30]:
Return_variable.describe()

,gvkey,fyear,LOGSIZE,B/M,Winsorised BM,Leverage,Winsorised Leverage,INVEST/A,Winsorised INVESTA,ROE,Winsorised ROE,LOGPPE,Winsorised SALESGR
count,16131.000000,16131.000000,15337.000000,15312.000000,15312.000000,16078.000000,16078.000000,16083.000000,16083.000000,16115.000000,16115.000000,13526.000000,13494.000000
mean,55550.154051,2016.565495,8.014688,0.530916,0.522397,0.281535,0.277260,0.034227,0.033341,11.378577,6.274868,6.760148,0.113642
std,64313.238997,2.859595,1.792287,0.751260,0.445978,0.238564,0.215525,0.044607,0.039312,525.150468,46.942138,2.240567,2.047087
min,1004.000000,2010.000000,0.364087,-24.427128,-0.218942,0.000000,0.000000,-0.185768,0.000000,-10404.347826,-240.205070,-2.465104,-0.749541
25%,9667.000000,2015.000000,6.759701,0.219071,0.219071,0.093193,0.093193,0.005686,0.005686,2.271908,2.271908,5.357308,-0.017796
50%,24344.000000,2017.000000,8.033157,0.419836,0.419836,0.253021,0.253021,0.021055,0.021055,9.641666,9.641666,6.836329,0.051554
75%,66708.000000,2019.000000,9.207703,0.714851,0.714851,0.412144,0.412144,0.045433,0.045433,17.534242,17.534242,8.236327,0.136167
max,328795.000000,2020.000000,14.491552,30.468414,2.344154,4.942947,0.904107,0.605523,0.203190,38870.000000,186.515669,13.144907,225.869455


### Calculate variables which use weekly_equity_return

In [31]:
data_raw = pd.read_csv('./dataset_raw_v2.csv')
weekly_equity_returns = pd.read_csv('./weekly_equity_returns.csv')
total_return_weekly = pd.read_csv('./total_returns_weekly.csv')

In [32]:
weekly_equity_returns_new = pd.merge(weekly_equity_returns, total_return_weekly[['Date', 'Total Return', 'GICS Sector Name', 'Company Common Name', 'gvkey', 'eow']], on=['gvkey', 'eow'], how='left')

In [33]:
weekly_equity_returns_new.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'eow', 'Instrument', 'Close_Price',
       'Bid_Price', 'Ask_Price', 'Volume', 'Company_Market_Cap', 'CF_CURR',
       'GICS_Sector_Name', 'Company_Common_Name', 'gvkey', 'mrg', 'X_USD',
       'Date', 'Total Return', 'GICS Sector Name', 'Company Common Name'],
      dtype='object')

In [34]:
# get gvkey in data_raw
companies_with_emission_data = data_raw['gvkey'].unique()

# get gvkey in weekly equity returns
weekly_equity_returns_filtered = weekly_equity_returns_new[weekly_equity_returns_new['gvkey'].isin(companies_with_emission_data)]
weekly_equity_returns_filtered.sort_values(by = ['gvkey', 'eow'])

weekly_equity_returns_filtered['eow'] = pd.to_datetime(weekly_equity_returns_filtered['eow'])
weekly_equity_returns_filtered['mom_eow'] = pd.to_datetime(weekly_equity_returns_filtered['eow']).copy()
weekly_equity_returns_filtered.set_index('eow', inplace=True)

C:\Users\13976\AppData\Local\Temp\ipykernel_29928\51203113.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weekly_equity_returns_filtered['eow'] = pd.to_datetime(weekly_equity_returns_filtered['eow'])
C:\Users\13976\AppData\Local\Temp\ipykernel_29928\51203113.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weekly_equity_returns_filtered['mom_eow'] = pd.to_datetime(weekly_equity_returns_filtered['eow']).copy()


### Calculate RET%

In [35]:
# Resample the data by month and get the data of last day in every month
monthly_data = weekly_equity_returns_filtered.resample('M').last()
Return_variable_RET = monthly_data[['gvkey']].copy()

In [36]:
# Group by 'gvkey' and 'eow' and calculate the closing price on the last day of the month
monthly_last_price = weekly_equity_returns_filtered.groupby(['gvkey', pd.Grouper(freq='M')])['Close_Price'].last()

# Calculate monthly return
monthly_return = monthly_last_price.groupby(level=0).pct_change()

# Reset index to get 'gvkey' and 'eow'
monthly_return = monthly_return.reset_index()

# Transform 'eow' to month type
monthly_return['month'] = monthly_return['eow'].dt.to_period('M')

# Rename 'Close_Price' as 'RET'
monthly_return.rename(columns={'Close_Price': 'RET'}, inplace=True)
monthly_return.dropna(inplace=True)

# Calculate the number of rows should be replaced
num_rows_to_replace1 = int(len(monthly_return) * 0.0025)
num_rows_to_replace2 = int(len(monthly_return) * 0.0025)

# Sort and replace the extreme values
monthly_return_sorted = monthly_return.sort_values(by='RET', ascending=True)
if num_rows_to_replace1 > 0 and num_rows_to_replace2 > 0:
    monthly_return_sorted.iloc[:num_rows_to_replace2, monthly_return_sorted.columns.get_loc('RET')] = np.nan
    monthly_return_sorted.iloc[-num_rows_to_replace1:, monthly_return_sorted.columns.get_loc('RET')] = np.nan

# Create new dataframe
Return_variable_RET = monthly_return_sorted.copy()

# Drop NaN
Return_variable_RET.dropna(subset=['RET'], inplace=True)
Return_variable_RET['RET%'] = Return_variable_RET['RET'] * 100

### Calculate MOM

In [37]:
Return_variable_MOM = pd.DataFrame(weekly_equity_returns_filtered['gvkey'])

In [38]:
# Transform 'eow' to datetime and add year, month as columns
weekly_equity_returns_filtered['mom_eow'] = pd.to_datetime(weekly_equity_returns_filtered['mom_eow'])
weekly_equity_returns_filtered['fyear'] = weekly_equity_returns_filtered['mom_eow'].dt.year
weekly_equity_returns_filtered['month'] = weekly_equity_returns_filtered['mom_eow'].dt.month

# Calculate the close price of every company in month
monthly_prices = weekly_equity_returns_filtered.groupby(['gvkey', 'fyear', 'month'])['Close_Price'].last().reset_index()

# Sort
monthly_prices.sort_values(by=['gvkey', 'fyear', 'month'], inplace=True)

# Construct yearly window
monthly_prices['Close_Price_1y_ago'] = monthly_prices.groupby('gvkey')['Close_Price'].shift(12)

# Calculate MOM
monthly_prices['MOM'] = monthly_prices['Close_Price'] / monthly_prices['Close_Price_1y_ago'] - 1

# Renew Return_variable DataFrame
Return_variable_MOM = monthly_prices[['gvkey', 'fyear', 'month', 'MOM']]

C:\Users\13976\AppData\Local\Temp\ipykernel_29928\669015328.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weekly_equity_returns_filtered['mom_eow'] = pd.to_datetime(weekly_equity_returns_filtered['mom_eow'])
C:\Users\13976\AppData\Local\Temp\ipykernel_29928\669015328.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weekly_equity_returns_filtered['fyear'] = weekly_equity_returns_filtered['mom_eow'].dt.year
C:\Users\13976\AppData\Local\Temp\ipykernel_29928\669015328.py:4: SettingWithCopyWarning: 
A v

In [39]:
# Drop NaN
MOM1 = Return_variable_MOM[['gvkey', 'fyear','month', 'MOM']].dropna()

# Winsorised the data in 2.5%
winsorised_MOM = winsorize(MOM1['MOM'], limits=[0.0025, 0.0025])

# Add winsorised data
MOM1['Winsorised MOM'] = winsorised_MOM

In [40]:
# Merge winsorised data to emission table on 'gvkey' and 'fyear'
Return_variable_MOM = Return_variable_MOM.merge(
    MOM1[['gvkey', 'fyear', 'month', 'Winsorised MOM']],
    on=['gvkey', 'fyear', 'month'],
    how='left'
)

In [41]:
Return_variable_MOM['MOM_duplicate'] = Return_variable_MOM['Winsorised MOM'].drop_duplicates()

### Calculate Beta

In [42]:
betas = pd.read_csv('./betas.csv')

### Calculate Voltility

In [43]:
weekly_equity_returns_filtered.reset_index()

# Calculate the monthly return
monthly_returns = weekly_equity_returns_filtered.groupby('gvkey').resample('M')['Close_Price'].last().pct_change()

# Reset index to get 'eow' and 'gvkey'
monthly_returns = monthly_returns.reset_index()

In [44]:
volat = pd.DataFrame({
    'gvkey': weekly_equity_returns_filtered['gvkey'],
    'mom_eow': weekly_equity_returns_filtered['mom_eow']
})
volat = volat.reset_index()

In [45]:
monthly_volat = monthly_returns.groupby('gvkey').apply(lambda x: x.set_index('eow')['Close_Price'].rolling(window=12).std())

In [46]:
volat = volat.join(monthly_volat, on=['gvkey', 'eow'], rsuffix='_volat')
volat.rename(columns={'Close_Price': 'VOLAT'}, inplace=True)

In [47]:
volat_unique = volat.drop_duplicates()

In [48]:
# Drop NaN
volat_unique.dropna(subset=['VOLAT'], inplace=True)

# Calculate the number of rows shoule be replaced
num_rows_to_replace = int(len(volat_unique) * 0.0025)

# Ensure at least one row has been replaced
if num_rows_to_replace < 1 and len(volat_unique) > 0:
    num_rows_to_replace = 1

# Sort and replace the extreme values
if num_rows_to_replace > 0:
    # Sort 'VOLAT'
    volat_sorted = volat_unique['VOLAT'].sort_values()

    lower_bound = volat_sorted.iloc[num_rows_to_replace - 1]
    upper_bound = volat_sorted.iloc[-num_rows_to_replace]
    volat_unique['VOLAT'] = volat_unique['VOLAT'].clip(lower=lower_bound, upper=upper_bound)

C:\Users\13976\AppData\Local\Temp\ipykernel_29928\1945059889.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volat_unique.dropna(subset=['VOLAT'], inplace=True)
C:\Users\13976\AppData\Local\Temp\ipykernel_29928\1945059889.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volat_unique['VOLAT'] = volat_unique['VOLAT'].clip(lower=lower_bound, upper=upper_bound)


### Calculate EPSGR

In [49]:
# Load data
EPS_data = pd.read_csv('./eps_marketLeverage.csv')

# Filter out the gvkey contained in companies_with_emission_data
EPS_data_filtered = EPS_data[EPS_data['gvkey'].isin(companies_with_emission_data)]

# Create Return_variable_EPSGR DataFrame
Return_variable_EPSGR = pd.DataFrame(EPS_data_filtered[['gvkey', 'fyear']])

# Sort
EPS_data_filtered = EPS_data_filtered.sort_values(by=['gvkey', 'fyear'])

# Calculate previous EPS
Return_variable_EPSGR['EPSGR'] = EPS_data_filtered.groupby('gvkey')['EPS'].pct_change()
Return_variable_EPSGR['EPSGR'].replace(float('inf'), np.nan, inplace=True)
Return_variable_EPSGR['fyear'] = EPS_data_filtered['fyear']

In [50]:
Return_variable_EPSGR['EPSGR'] = Return_variable_EPSGR['EPSGR'].drop_duplicates()
Return_variable_EPSGR['EPSGR'].replace([np.inf, -np.inf], np.nan, inplace=True)

In [51]:
Return_variable_EPSGR['EPSGR'].dropna(inplace=True)

# First, filter out the rows with non-null EPSGR values
epsgr_non_null = Return_variable_EPSGR[Return_variable_EPSGR['EPSGR'].notna()]

# Calculate 1.25% of the number of rows in the non-null data
num_rows_to_replace_eps1 = int(len(epsgr_non_null) * 0.015)
num_rows_to_replace_eps2 = int(len(epsgr_non_null) * 0.025)

# Sort the non-null data
epsgr_non_null_sorted = epsgr_non_null.sort_values(by='EPSGR', ascending=True)

# Get the index of the EPSGR column
epsgr_col_index = epsgr_non_null_sorted.columns.get_loc('EPSGR')

# Remove the smallest 1.25% of the data
epsgr_non_null_sorted.iloc[:num_rows_to_replace_eps2, epsgr_col_index] = np.nan

# Remove the largest 1.25% of the data
epsgr_non_null_sorted.iloc[-num_rows_to_replace_eps1:, epsgr_col_index] = np.nan

# Re-merge into the original DataFrame
Return_variable_EPSGR.loc[epsgr_non_null_sorted.index, 'EPSGR'] = epsgr_non_null_sorted['EPSGR']

Return_variable_EPSGR['EPSGR'] = Return_variable_EPSGR['EPSGR'].drop_duplicates()

## Merge all data to one dataframe

### Merge part of return variable to the main dataframe

In [52]:
Emission_table['gvkey'] = Emission_table['gvkey'].astype('float64')
Emission_table['fyear'] = Emission_table['fyear'].astype('float64')

In [53]:
Dataframe1 = pd.merge(Emission_table, Return_variable, on=['gvkey','fyear'], how='left')

In [54]:
Dataframe1.describe()

,gvkey,fyear,log_scope_1,log_scope_2,Winsorised Growth Rate in Carbon Emissions Scope 1,Winsorised Growth Rate in Carbon Emissions Scope 2,Carbon Intensity Scope 1,Winsorised Carbon Intensity Scope 1,Carbon Intensity Scope 2,Winsorised Carbon Intensity Scope 2,...,B/M,Winsorised BM,Leverage,Winsorised Leverage,INVEST/A,Winsorised INVESTA,ROE,Winsorised ROE,LOGPPE,Winsorised SALESGR
count,17787.000000,17787.000000,17780.000000,17782.000000,15161.000000,15161.000000,17433.000000,17433.000000,17433.000000,17433.000000,...,16824.000000,16824.000000,17734.000000,17734.000000,17739.000000,17739.000000,17771.000000,17771.000000,14694.000000,15134.000000
mean,53230.561590,2016.526339,10.010713,10.236393,0.067578,0.086422,1.613004,1.385161,0.361470,0.314770,...,0.537935,0.530187,0.276610,0.272734,0.033476,0.032673,11.357607,6.729490,6.926969,0.104371
std,63334.284067,2.860865,3.021521,2.547694,0.375825,0.404787,7.029082,5.023132,1.624064,0.425497,...,0.726927,0.442590,0.232138,0.210634,0.044757,0.039990,500.089011,44.795713,2.280663,1.933412
min,1004.000000,2010.000000,-1.645065,-0.079043,-0.766253,-0.676186,0.000000,0.002459,0.000000,0.006984,...,-24.427128,-0.218942,0.000000,0.000000,-0.185768,0.000000,-10404.347826,-240.205070,-2.465104,-0.749541
25%,8402.000000,2015.000000,7.974190,8.708996,-0.065107,-0.067892,0.035993,0.035993,0.075461,0.075461,...,0.229754,0.229754,0.098418,0.098418,0.005145,0.005145,2.839012,2.839012,5.453996,-0.012607
50%,22119.000000,2017.000000,9.975233,10.400579,0.011421,0.009264,0.130036,0.130036,0.170307,0.170307,...,0.420942,0.420942,0.247761,0.247761,0.019483,0.019483,9.561149,9.561149,6.995043,0.043912
75%,66210.000000,2019.000000,12.028111,12.111406,0.123437,0.147985,0.294071,0.294071,0.399564,0.399564,...,0.734708,0.734708,0.406605,0.406605,0.043475,0.043475,17.581344,17.581344,8.558613,0.128715
max,328795.000000,2020.000000,18.804073,16.920606,2.102875,2.270040,196.809468,34.648542,141.591432,2.519237,...,30.468414,2.344154,4.942947,0.904107,0.605523,0.203190,38870.000000,186.515669,13.144907,225.869455


### Let RET to be the main dataframe

In [55]:
Return_variable_RET['fyear'] = Return_variable_RET['eow'].dt.year
Return_variable_RET['month'] = Return_variable_RET['eow'].dt.month

In [56]:
Main_dataframe1 = pd.merge(Return_variable_RET, Dataframe1, on=['gvkey','fyear'], how='left')

### Merge MOM to the main dataframe

In [57]:
Main_dataframe2 = pd.merge(Main_dataframe1, Return_variable_MOM, on=['gvkey','fyear','month'], how='left')

### Merge beta to the main dataframe

In [58]:
betas['eow'] = pd.to_datetime(betas['eow'])
betas['fyear'] = betas['eow'].dt.year
betas['month'] = betas['eow'].dt.month

In [59]:
monthly_beta = betas.groupby(['gvkey','fyear','month'])['beta'].mean().reset_index()

In [60]:
Main_dataframe3 = pd.merge(Main_dataframe2, monthly_beta, on=['gvkey','fyear','month'], how='left')

### Merge volatility to the main dataframe

In [61]:
volat_unique['eow'] = pd.to_datetime(volat_unique['eow'])
volat_unique['fyear'] = volat_unique['eow'].dt.year
volat_unique['month'] = volat_unique['eow'].dt.month
# volat_month = volat_unique.groupby(['gvkey','fyear','month'])['VOLAT'].mean()

C:\Users\13976\AppData\Local\Temp\ipykernel_29928\3429661774.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volat_unique['eow'] = pd.to_datetime(volat_unique['eow'])
C:\Users\13976\AppData\Local\Temp\ipykernel_29928\3429661774.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  volat_unique['fyear'] = volat_unique['eow'].dt.year
C:\Users\13976\AppData\Local\Temp\ipykernel_29928\3429661774.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

In [62]:
Main_dataframe4 = pd.merge(Main_dataframe3, volat_unique, on=['gvkey','fyear','month'], how='left')

### Merge EPSGR to the main dataframe

In [63]:
Main_dataframe5 = pd.merge(Main_dataframe4, Return_variable_EPSGR, on=['gvkey','fyear'], how='left')

## Get industry from data_raw

In [64]:
data_raw['gvkey'] = data_raw['gvkey'].astype(str)
data_raw['fyear'] = data_raw['fyear'].astype(int).astype(str)

In [65]:
Main_dataframe5['gvkey'] = Main_dataframe5['gvkey'].astype(int).astype(str)
Main_dataframe5['fyear'] = Main_dataframe5['fyear'].astype(int).astype(str)

In [66]:
# Check if gvkey in Dataframe3 exists in data_raw
data_raw_gvkeys = set(data_raw['gvkey'])
df3_gvkeys = set(Main_dataframe5['gvkey'])

missing_gvkeys = df3_gvkeys - data_raw_gvkeys
print(f"missiong gvkey: {missing_gvkeys}")

missiong gvkey: set()


In [67]:
# If no missing gvkeys, merge the dataframe
if len(missing_gvkeys) == 0:
    merged_df = pd.merge(Main_dataframe5, data_raw[['gvkey','fyear','GICS_level_1']], on=['gvkey', 'fyear'], how='left')
else:
    print("wrong")

In [68]:
# Check duplicated values
duplicates = merged_df['gvkey'].duplicated()
has_duplicates = duplicates.any()

In [69]:
merged_df = merged_df.drop(columns=['RET', 'Carbon Intensity Scope 1', 'Carbon Intensity Scope 2', 'B/M', 'Leverage', 'INVEST/A', 'ROE', 'MOM', 'Winsorised MOM'])

In [70]:
merged_df.describe()

,month,RET%,log_scope_1,log_scope_2,Winsorised Growth Rate in Carbon Emissions Scope 1,Winsorised Growth Rate in Carbon Emissions Scope 2,Winsorised Carbon Intensity Scope 1,Winsorised Carbon Intensity Scope 2,LOGSIZE,Winsorised BM,Winsorised Leverage,Winsorised INVESTA,Winsorised ROE,LOGPPE,Winsorised SALESGR,MOM_duplicate,beta,VOLAT,EPSGR
count,385419.000000,385419.000000,231039.000000,231064.000000,203232.000000,203232.000000,226999.000000,226999.000000,218482.000000,218208.000000,230520.000000,230590.000000,230932.000000,185867.000000,202652.000000,337880.000000,362377.000000,48965.000000,230943.000000
mean,6.554573,0.982939,10.196407,10.469592,0.063291,0.083948,1.263314,0.317250,8.428594,0.546065,0.270907,0.029647,7.711958,7.268571,0.090334,0.135931,1.105152,0.114619,0.025154
std,3.438300,11.314887,3.124783,2.718884,0.370854,0.413537,4.693970,0.423105,1.990615,0.434432,0.202770,0.036277,41.596463,2.336983,1.797808,0.478318,0.625873,0.157452,1.784203
min,1.000000,-46.052632,-1.645065,-0.079043,-0.766253,-0.676186,0.002459,0.006984,0.364087,-0.218942,0.000000,0.000000,-240.205070,-2.465104,-0.749541,-0.888750,-0.171023,0.023020,-7.513988
25%,4.000000,-4.822899,8.036567,8.764322,-0.060508,-0.062660,0.039278,0.077062,6.931848,0.255590,0.104167,0.003787,3.655294,5.711029,-0.008758,-0.123426,0.695100,0.060661,-0.550468
50%,7.000000,0.772200,10.186898,10.688325,0.000253,0.000000,0.130756,0.175473,8.315117,0.429435,0.249756,0.018633,9.838676,7.356216,0.033179,0.074738,1.055753,0.085914,-0.015109
75%,10.000000,6.319703,12.666913,12.793022,0.106248,0.132487,0.295027,0.401121,10.036651,0.762309,0.395625,0.039741,18.272460,9.078945,0.114588,0.298988,1.444517,0.126845,0.423139
max,12.000000,69.826889,18.804073,16.920606,2.102875,2.270040,34.648542,2.519237,14.491552,2.344154,0.904107,0.203190,186.515669,13.144907,225.869455,4.711340,2.832603,2.681848,12.620800


In [ ]:
merged_df.to_csv('Regression_month_new.csv', index=True)